In [1]:
import numpy as np
import pandas as pd
%matplotlib inline

	- instant: record index
	- dteday : date
	- season : season (1:springer, 2:summer, 3:fall, 4:winter)
	- yr : year (0: 2011, 1:2012)
	- mnth : month ( 1 to 12)
	- hr : hour (0 to 23)
	- holiday : weather day is holiday or not (extracted from http://dchr.dc.gov/page/holiday-schedule)
	- weekday : day of the week
	- workingday : if day is neither weekend nor holiday is 1, otherwise is 0.
	+ weathersit : 
		- 1: Clear, Few clouds, Partly cloudy, Partly cloudy
		- 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
		- 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
		- 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
	- temp : Normalized temperature in Celsius. The values are divided to 41 (max)
	- atemp: Normalized feeling temperature in Celsius. The values are divided to 50 (max)
	- hum: Normalized humidity. The values are divided to 100 (max)
	- windspeed: Normalized wind speed. The values are divided to 67 (max)
	- casual: count of casual users
	- registered: count of registered users
	- cnt: count of total rental bikes including both casual and registered

In [2]:
# Read in data
df = pd.read_csv('~/Desktop/py/data/day.csv')
df.shape

(731, 16)

In [3]:
from sklearn.model_selection import train_test_split as tts

features = [
    'instant', 'season', 'yr', 'mnth', 'holiday', 'weekday',
    'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed'
]

target = 'cnt'

X = df[features]
y = df[target]

X_train, X_test, y_train, y_test = tts(X, y, test_size=0.2)

In [4]:
print('X_train shape: ', X_train.shape,'y_train shape: ', y_train.shape)

X_train shape:  (584, 12) y_train shape:  (584,)


In [ ]:
'''# Preprocessing - DataFrameMapper is acting up so using https://stackoverflow.com/questions/37685412/avoid-scaling-binary-columns-in-sci-kit-learn-standsardscaler
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

continuous_cols = [
    'instant', 'season', 'mnth', 'weekday',
    'weathersit', 'temp', 'atemp', 'hum', 'windspeed'
]

binary_cols = ['yr', 'holiday', 'workingday']

mapper = DataFrameMapper(
    [(continuous_col, StandardScaler()) for continuous_col in continuous_cols] +
    [(binary_cols, None) for binary_col in binary_cols]
)

pipeline = Pipeline([
    ('mapper', mapper)
])

pipeline.fit_transform(X_train)
X_train.describe()'''

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin

class StandardScalerSelect(BaseEstimator, TransformerMixin): 
    '''
    This class enables us to choose which columns to StandardScale.
    Only the variables referred to with cont_vars_index are scaled.
    '''
    def __init__(self, bin_vars_index, cont_vars_index, copy=True, with_mean=True, with_std=True):
        self.scaler = StandardScaler(copy, with_mean, with_std)
        self.bin_vars_index = bin_vars_index
        self.cont_vars_index = cont_vars_index

    def fit(self, X, y=None):
        self.scaler.fit(X[:, self.cont_vars_index], y)
        return self

    def transform(self, X, y=None, copy=None):
        X_tail = self.scaler.transform(X[:, self.cont_vars_index], y, copy)
        return np.concatenate((X[:, self.bin_vars_index], X_tail), axis=1)

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

cont_vars = [
    'instant', 'season', 'mnth', 'weekday',
    'weathersit', 'temp', 'atemp', 'hum', 'windspeed'
]

bin_vars = ['yr', 'holiday', 'workingday']

cont_vars_index = [X.columns.get_loc(c) for c in X.columns if c in cont_vars]

bin_vars_index = [X.columns.get_loc(c) for c in X.columns if c in bin_vars]

# Unsure about what order to do the scaling and polynomial expansion in

model = Pipeline([
    ('poly', PolynomialFeatures(2)),
    ('scaler', StandardScalerSelect(bin_vars_index=bin_vars_index, cont_vars_index=cont_vars_index)),
])

X_train_new = model.fit_transform(X_train, y_train)

c:\econsw\python36\lib\site-packages\sklearn\preprocessing\data.py:675: DeprecationWarning: The parameter y on transform() is deprecated since 0.19 and will be removed in 0.21
  DeprecationWarning)


Not sure what to do from here. Adding a model to the end of pipeline? Ran into some issues trying to combine the custom scaler and LassoCV.